In [ ]:
from tensorflow.keras import Model, layers, activations
import tensorflow_addons as tfa
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [ ]:
!pip install tensorflow_addons

# Set up the TPU

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Running on TPU",tpu.master())
except:
    tpu=None
    print("Nope")
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
print("Replicas",strategy.num_replicas_in_sync)

# Defining Model

In [ ]:
from tensorflow.keras import Model, layers, activations
import tensorflow_addons as tfa
import math

def round_filters(filters, multiplier=1.):
    divisor = 8
    min_depth = 8
    filters *= multiplier
    min_depth = min_depth or divisor
    new_filters = max(min_depth, int(filters + divisor / 2) // divisor * divisor)
    return int(new_filters)


def round_repeats(repeats, multiplier=1.):
    return int(math.ceil(multiplier * repeats))


def squeeze_and_excite(x, in_channels, out_channels, activation, reduction_ratio=4):
    x = layers.GlobalAvgPool2D()(x)
    x = layers.Dense(in_channels // reduction_ratio)(x)
    x = layers.Activation(activation)(x)
    x = layers.Dense(out_channels)(x)
    x = layers.Activation(activations.sigmoid)(x)
    return x


def fused_mbconv(x, in_channels, out_channels, kernel_size, activation, stride=1, reduction_ratio=4,
                 expansion=6, dropout=None, drop_connect=.2):
    shortcut = x
    expanded = round_filters(in_channels * expansion)

    if expansion != 1:
        x = layers.Conv2D(expanded, kernel_size, stride, padding="same", use_bias=False)(x)
        x = layers.BatchNormalization(epsilon=1e-5)(x)
        x = layers.Activation(activation)(x)

        if (dropout is not None) and (dropout != 0.):
            x = layers.Dropout(dropout)(x)

    if reduction_ratio is not None:
        se = squeeze_and_excite(x, in_channels, expanded, activation, reduction_ratio)
        x = layers.Multiply()([x, se])

    x = layers.Conv2D(out_channels, (1, 1) if expansion != 1 else kernel_size, 1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    if expansion == 1:
        x = layers.Activation(activation)(x)
    if (stride == 1) and (in_channels == out_channels):
        x = tfa.layers.StochasticDepth(1 - drop_connect)([shortcut, x])
    return x


def mbconv(x, in_channels, out_channels, kernel_size, activation, stride=1,
           reduction_ratio=4, expansion=6, dropout=None, drop_connect=.2):
    shortcut = x
    expanded = round_filters(in_channels * expansion)

    if expansion != 1:
        x = layers.Conv2D(expanded, (1, 1), 1, padding="same", use_bias=False)(x)
        x = layers.BatchNormalization(epsilon=1e-5)(x)
        x = layers.Activation(activation)(x)

    x = layers.DepthwiseConv2D(kernel_size=kernel_size, strides=stride, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    x = layers.Activation(activation)(x)

    if (expansion != 1) and (dropout is not None) and (dropout != 0.):
        x = layers.Dropout(dropout)(x)

    if reduction_ratio is not None:
        se = squeeze_and_excite(x, in_channels, expanded, activation, reduction_ratio)
        x = layers.Multiply()([x, se])

    x = layers.Conv2D(out_channels, (1, 1), 1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    if (stride == 1) and (in_channels == out_channels):
        x = tfa.layers.StochasticDepth(1 - drop_connect)([shortcut, x])
    return x


def repeat(x, count, in_channels, out_channels, kernel_size, activation,
           stride=1, reduction_ratio=None, expansion=6, fused=False, dropout=None, drop_connect=.2):
    for i in range(count):
        if fused:
            x = fused_mbconv(x, in_channels, out_channels, kernel_size,
                             activation, stride, reduction_ratio, expansion, dropout, drop_connect)
        else:
            x = mbconv(x, in_channels, out_channels, kernel_size, activation, stride,
                       reduction_ratio, expansion, dropout, drop_connect)
    return x


def stage(x, count, in_channels, out_channels, kernel_size, activation,
          stride=1, reduction_ratio=None, expansion=6, fused=False, dropout=None, drop_connect=.2):
    x = repeat(x, count=1, in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
               activation=activation, stride=stride, reduction_ratio=reduction_ratio,
               expansion=expansion, fused=fused, dropout=dropout, drop_connect=drop_connect)
    x = repeat(x, count=count - 1, in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size,
               activation=activation, stride=1, reduction_ratio=reduction_ratio,
               expansion=expansion, fused=fused, dropout=dropout, drop_connect=drop_connect)
    return x


def base(cfg, num_classes=5, input_tensor=None, activation=activations.swish,
         width_mult=1., depth_mult=1., conv_dropout_rate=None, dropout_rate=None, drop_connect=.2):
    inp = input_tensor
    # stage 0
    x = layers.Conv2D(cfg[0][4], kernel_size=(3, 3), strides=2, padding="same", use_bias=False)(inp)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    x = layers.Activation(activation)(x)

    for stage_cfg in cfg:
        x = stage(x, count=round_repeats(stage_cfg[0], depth_mult),
                  in_channels=round_filters(stage_cfg[4], width_mult),
                  out_channels=round_filters(stage_cfg[5], width_mult),
                  kernel_size=stage_cfg[1], activation=activation, stride=stage_cfg[2],
                  reduction_ratio=stage_cfg[7], expansion=stage_cfg[3], fused=stage_cfg[6] == 1,
                  dropout=conv_dropout_rate, drop_connect=drop_connect)

    # final stage
    x = layers.Conv2D(round_filters(1280, width_mult), (1, 1), strides=1, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    x = layers.Activation(activation)(x)

    x = layers.GlobalAvgPool2D()(x)
    if (dropout_rate is not None) and (dropout_rate != 0):
        x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(num_classes)(x)
    x = layers.Activation(activations.softmax)(x)

    return Model(inp, x)


def s(in_shape=(224, 224, 3), num_classes=7, input_tensor=None, activation=activations.swish,
      width_mult=1., depth_mult=1., conv_dropout_rate=None, dropout_rate=None, drop_connect=.2):
    # each row is a stage
    # count, kernel size, stride, expansion ratio, in channel, out channel, is fused(1 if true), reduction ratio(None if no se)
    cfg = [
        [2, 3, 1, 1, 24, 24, 1, None],
        [4, 3, 2, 4, 24, 48, 1, None],
        [4, 3, 2, 4, 48, 64, 1, None],
        [6, 3, 2, 4, 64, 128, 0, 4],
        [9, 3, 1, 6, 128, 160, 0, 4],
        [15, 3, 2, 6, 160, 256, 0, 4],
    ]
    input_tensor = layers.Input(in_shape) if input_tensor is None else input_tensor
    return base(cfg=cfg, num_classes=num_classes, input_tensor=input_tensor, activation=activation,
                width_mult=width_mult, depth_mult=depth_mult, conv_dropout_rate=conv_dropout_rate,
                dropout_rate=dropout_rate, drop_connect=drop_connect)

def xl(in_shape=(224, 224, 3), num_classes=4, input_tensor=None, activation=activations.swish,
      width_mult=1.0, depth_mult=1., conv_dropout_rate=None, dropout_rate=None, drop_connect=.2):
    cfg = [
        [4, 3, 1, 1, 32, 32, 1, None],
        [8, 3, 2, 4, 32, 64, 1, None],
        [8, 3, 2, 4, 64, 96, 1, None],
        [16, 3, 2, 4, 96, 192, 0, 4],
        [24, 3, 1, 6, 192, 256, 0, 4],
        [32, 3, 2, 6, 256, 512, 0, 4],
        [8, 3, 1, 6, 512, 640, 0, 4],
    ]
    input_tensor = layers.Input(in_shape) if input_tensor is None else input_tensor
    return base(cfg=cfg, num_classes=num_classes, input_tensor=input_tensor, activation=activation,
                width_mult=width_mult, depth_mult=depth_mult, conv_dropout_rate=conv_dropout_rate,
                dropout_rate=dropout_rate, drop_connect=drop_connect)

# Defining parameters

In [ ]:
IMAGE_SIZE = [300, 300] # at this size, a GPU will run out of memory. Use the TPU
EPOCHS = 150
BATCH_SIZE = 32 * strategy.num_replicas_in_sync
NUM_TRAINING_IMAGES = 11932
NUM_VAL_IMAGES = 3976
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VAL_STEPS = NUM_VAL_IMAGES // BATCH_SIZE

# Loading the Dataset

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('Tfrec1')

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3])# explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "label": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['label'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.
    AUTO = tf.data.experimental.AUTOTUNE
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.prefetch(AUTO)
    dataset = dataset.map(read_labeled_tfrecord,num_parallel_calls=AUTO) # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    c=1
    plt.figure(figsize=(20, 4))
    for features in dataset.take(5):
        plt.subplot(1,5,c)
        plt.imshow(features[0].numpy())
        c+=1
    plt.show()
    return dataset

def get_training_dataset():
    filenames = tf.io.gfile.glob(GCS_DS_PATH + '/Tfrec1/train/*.tfrec')
    dataset = load_dataset(filenames, labeled=True, ordered=False)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(NUM_TRAINING_IMAGES)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def get_validation_dataset():
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/Tfrec1/val/*.tfrec'), labeled=True, ordered=False)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    return dataset

def get_test_dataset(ordered=True):
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/Tfrec1/test/*.tfrec'), labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()
test_dataset = get_test_dataset()

# Training the Model

In [ ]:
# instantiating the model in the strategy scope creates the model on the TPU
with strategy.scope():
    model = s(in_shape=(300, 300, 3),num_classes=5, conv_dropout_rate=0.7, dropout_rate=0.7)
    model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

history = model.fit(training_dataset,steps_per_epoch=STEPS_PER_EPOCH,validation_data=validation_dataset,epochs=EPOCHS,validation_steps=VAL_STEPS)

In [ ]:

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 5])
plt.legend(loc='lower right')

plt.figure(2)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

#test_array = np.array(list(test_dataset.unbatch().take(-1).as_numpy_iterator()), dtype=object)
#test_x = np.stack(test_array[:,0])
#test_y = np.stack(test_array[:,1])

# Use the model to predict the labels
#test_predictions = model.predict(test_x)
#test_y_pred = np.argmax(test_predictions, axis=1)
#test_y_true = np.argmax(test_y, axis=1)

# Evaluating model accuracy and logging it as a scalar for TensorBoard hyperparameter visualization.
#accuracy = sklearn.metrics.accuracy_score(test_y_true, test_y_pred)
#tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
#logging.info('Test accuracy:{}'.format(accuracy))

In [ ]:
import pickle

with open('./efficientnettrainHistory-goodcdr0.7', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [ ]:
history = pickle.load(open('./efficientnettrainHistory-goodcdr0.7', "rb"))

In [ ]:

plt.plot(history['loss'], label='loss')
plt.plot(history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 5])
plt.legend(loc='lower right')

plt.figure(2)
plt.plot(history['accuracy'], label='accuracy')
plt.plot(history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
class_names = ['Acne','Eczema','Melanoma','Psoriasis']
url = "https://images.prismic.io/npf-website/37fb7de7-c664-4112-8ae3-d26dcf6a75bb_plaque-psoriasis.jpg?auto=compress,format&rect=0,68,320,184&w=778&h=447"
path1 = tf.keras.utils.get_file('Red_sunflower', origin=url)

img1 = keras.preprocessing.image.load_img(
    path1, target_size=(300,300)
)
img_array = keras.preprocessing.image.img_to_array(img1)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(np.argmax(score), 100 * np.max(score))
)